In [ ]:
!pip install torch
!pip install transformers==2.1.1

In [ ]:
import os
# !wget https://convaisharables.blob.core.windows.net/lsp/multiref/large_ft.pkl
!wget https://convaisharables.blob.core.windows.net/lsp/multiref/medium_ft.pkl
  
if not os.path.exists("DialoGPT"):
  !git clone https://github.com/microsoft/DialoGPT.git

--2021-05-30 03:56:55--  https://convaisharables.blob.core.windows.net/lsp/multiref/medium_ft.pkl
Resolving convaisharables.blob.core.windows.net (convaisharables.blob.core.windows.net)... 13.77.184.64
Connecting to convaisharables.blob.core.windows.net (convaisharables.blob.core.windows.net)|13.77.184.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862954531 (823M) [application/octet-stream]
Saving to: ‘medium_ft.pkl.1’

medium_ft.pkl.1     100%[===================>] 822.98M  50.5MB/s    in 16s     

2021-05-30 03:57:12 (50.4 MB/s) - ‘medium_ft.pkl.1’ saved [862954531/862954531]



In [ ]:
import os
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
import numpy as np

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
# weights = torch.load('large_ft.pkl')
weights = torch.load('medium_ft.pkl')

# cfg = GPT2Config(n_embd=1024,n_layer=24,n_head=16)

cfg = GPT2Config.from_json_file('DialoGPT/configs/345M/config.json')
model = GPT2LMHeadModel(cfg)
# gpt2_medium_config = GPT2Config(n_ctx=1024, n_embd=1024, n_layer=24, n_head=16)
# model = GPT2LMHeadModel(gpt2_medium_config)

# fix misused key value
# weights["lm_head.weight"] = weights["lm_head.decoder.weight"]
# weights.pop("lm_head.decoder.weight",None)
# model.load_state_dict(weights)

model.load_state_dict(torch.load("medium_ft.pkl"), strict=False)
model.lm_head.weight.data = model.transformer.wte.weight.data # beg huggingface not to change this anymore

model.to('cuda')
model.eval()

conditioned_tokens = []
generated_tokens = []
past = None
fst = True
DELIMITER = "/n"

def reinput(user_msg):
	global conditioned_tokens; global fst
	os.system('cls' if os.name == 'nt' else 'clear')
	  
	conditioned_tokens = tokenizer.decode(conditioned_tokens)

	print("Me: " + user_msg + "\n" + "Bot: ",end='')
	
	if fst:
	  fst = False
	  user_msg = "<|endoftext|>" + user_msg
  
	user_msg = "" + user_msg
	conditioned_tokens += user_msg 
	conditioned_tokens = tokenizer.encode(conditioned_tokens) 
	conditioned_tokens += [50256] # Append operator to prepend conversation history
  


def top_p_filtering(logits, top_p=0.9, filter_value=-float('Inf')):
  """
  Credit: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
  """
  assert logits.dim() == 1  # batch size 1 for single word generation
  sorted_logits, sorted_indices = torch.sort(logits, descending=True)
  cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
  # remove tokens with cumulative probability above the threshold
  sorted_indices_to_remove = cumulative_probs > top_p
  # shift the indices to the right to keep also the first token above the threshold
  sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
  sorted_indices_to_remove[..., 0] = 0
  indices_to_remove = sorted_indices[sorted_indices_to_remove]
  logits[indices_to_remove] = filter_value
  return logits


def recalc(p=None):
  global conditioned_tokens
  global generated_tokens
  global past
  
  # for segment display purpose, keep 2 sets of tokens
  indexed_tokens = conditioned_tokens + generated_tokens
  
  tokens_tensor = torch.tensor([indexed_tokens])
  
  tokens_tensor = tokens_tensor.to('cuda')
  with torch.no_grad():
    outputs,past = model(tokens_tensor, past=p)
    # outputs,past = model(tokens_tensor, past_key_values=p) #  for transformers 4 #AK
    predictions = outputs
  logits = predictions[0, -1, :]
  filtered_logits = top_p_filtering(logits)
  probabilities = F.softmax(filtered_logits, dim=-1)
  next_token = torch.multinomial(probabilities, 1)
  generated_tokens.append(next_token.item())
  return next_token.item()

def generate():
	global conditioned_tokens
	global generated_tokens
	global past
 
	while True:
		if len(tokenizer.decode(conditioned_tokens)) > 320:
		  dc = tokenizer.decode(conditioned_tokens)
		  dc = dc[len(dc)-320:]
		  idx = dc.find(DELIMITER)
		  if idx != -1:
		    dc = dc[idx+len(DELIMITER):]
		    conditioned_tokens = tokenizer.encode(dc)
    
		result = recalc()

		if result == 50256:
      
      # end-of-text : 50256
      # use this special token to split segments

			decoded_reply = tokenizer.decode(generated_tokens)
      
			to_print = decoded_reply
			if to_print.endswith("<|endoftext|>"):
			  to_print = to_print[:-len("<|endoftext|>")]
			  decoded_reply = to_print
			print(to_print)
			
			decoded_reply = "" + decoded_reply
			decoded_reply = decoded_reply+DELIMITER
			conditioned_tokens += (tokenizer.encode(decoded_reply))
    
			# Remove end of text tokens from feeding back into model
      # -- see here for reason https://github.com/huggingface/transformers/issues/429#issuecomment-479380117
			cond_str = tokenizer.decode(conditioned_tokens)
			cond_str = cond_str.replace("<|endoftext|>",DELIMITER) # Since "<end-of-text> (50256)" is mapped to the newline character (\n) by default when calling decode():
			conditioned_tokens = tokenizer.encode(cond_str)
    
# Uncomment to debug (print) conversation history that is re-fed to the model
# 			cond_str = tokenizer.decode(conditioned_tokens)
# 			cond_str = cond_str.replace("<|endoftext_|>","\n").replace(DELIMITER,"\n") # Since "<end-of-text> (50256)" is mapped to the newline character (\n) by default when calling decode():
# 			print("(condstr_dbg) " + cond_str)
# 			print(r"(/constr_dbg)")
    
			generated_tokens = []
			break

100%|██████████| 456318/456318 [00:00<00:00, 1362810.43B/s]


How to use:

This script will automatically generate a complete segment(or a post if you wish) and wait for user input.
1. Press Enter (input nothing) to keep generating under current topic.
2. Type in prompt text to start another topic.

3. You can now type 'reset' to start the conversation over.

In [ ]:
from IPython.display import clear_output
import logging

def new_dialogue():

  global conditioned_tokens
  global generated_tokens
  global past; global fst
  
  conditioned_tokens = []; generated_tokens = []; past = None; fst = True
  reinput("What is the meaning of life?")
  generate()

  while True:
    cmd = input()

    if cmd == "reset":
      clear_output()
      new_dialogue()
      break

    if cmd != "":	
      reinput(cmd)
    generate()
    
new_dialogue()

Me: What is the meaning of life?
Bot: Life is like a bleached sponge.
What are you mean?
Me: What are you mean?
Bot: It's this similar to the phrase Beware of Blackberries?
No, it doen't
Me: No, it doen't
Bot: As the mathamicks above me points out, surely that isn't even a real word.
Do you see 13th floor movie?
Me: Do you see 13th floor movie?
Bot: Where's the footage from? I can't see it in the article.


KeyboardInterrupt: ignored